Import Statements

In [1]:
import os
import sys
import tensorflow as tf


# setting the root path
ROOT_DIR = os.path.join(os.getcwd(),"..","..")
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")
sys.path.append(ROOT_DIR)

from MetricTrackingCode import get_available_devices
from MetricTrackingCode import MTrackingDataset
from mrcnn.config import Config
from mrcnn import model as modellib, utils 

# setting the path to the mrcnn pre-trained weights
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR,"mrcnn", "mask_rcnn_coco.h5")

#checks if there are GPUs available
print(get_available_devices())

#show list of all GPU available
print("List of physical GPU devices")
gpu_devices = tf.config.list_physical_devices('GPU')
print(len(gpu_devices))
for gpu in gpu_devices:
    print("here")
    print(' '*3,gpu)
    try:
        tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

['/device:CPU:0']
List of physical GPU devices
0


SETTING THE CONFIGURATION FILE FOR TRAINING

In [2]:
class MTackingConfig(Config):
    Name = 'MTracking'
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 3
    EPOCHS = 20
    MAX_GT_INSTANCES = 1000
    DETECTION_MIN_CONFIDENCE = 0.9
    USE_MINI_MASK = False  
    RUN_EAGERLY = False
    MAX_QUEUE=10

config = MTackingConfig()
config.display()




Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
EPOCHS                         20
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                16
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE     

INITIALIZING THE DATAC CLASSES

In [3]:
MTracking_DIR = os.path.join(ROOT_DIR, "Dataset")

train_Dataset = MTrackingDataset()
val_Dataset = MTrackingDataset()

#Loading the data
train_Dataset.load_data("Dataset/train/labels/All_Annotations.json","Dataset/train")
val_Dataset.load_data("Dataset/train/labels/All_Annotations.json","Dataset/train")

#transfors data into container structure
train_Dataset.prepare()
val_Dataset.prepare()
config.VALIDATION_STEPS = len(val_Dataset.image_ids)

print("===========================================================")
print("Image Train Count: {}".format(len(train_Dataset.image_ids)))
print("Class Train Count: {}".format(train_Dataset.num_classes))
print("===========================================================")

for i, info in enumerate(train_Dataset.class_info):
      print("{:3}. {:50}".format(i, info['name']))
        
        
print("===========================================================")
print("Image val count: {}".format(len(val_Dataset.image_ids)))
print("Class val count: {}".format(val_Dataset.num_classes))
print("===========================================================")

for i, info in enumerate(val_Dataset.class_info):
    print("{:3}. {:50}".format(i, info['name']))

Dataset/train/labels/All_Annotations.json
<class 'dict'>
Dataset/train/labels/All_Annotations.json
<class 'dict'>
Image Train Count: 41
Class Train Count: 4
  0. BG                                                
  1. Root                                              
  2. Shoot                                             
  3. Node                                              
Image val count: 41
Class val count: 4
  0. BG                                                
  1. Root                                              
  2. Shoot                                             
  3. Node                                              


SETTING UP THE MODEL

In [4]:
model = modellib.MaskRCNN(mode="training",
                          config=config, 
                          model_dir=DEFAULT_LOGS_DIR)
print(COCO_WEIGHTS_PATH)
model.load_weights(COCO_WEIGHTS_PATH, by_name=True, 
                   exclude=["rpn_model",
                            "mrcnn_class_logits", 
                            "mrcnn_bbox_fc", 
                            "mrcnn_bbox", 
                            "mrcnn_mask"])

/home/ai/dev/Metric_Tracking/Mask_RCNN/samples/MetricTracking/../../mrcnn/mask_rcnn_coco.h5


TRAINING THE MODEL

In [ ]:

print("===== PRINT HERE ====")
print(config.EPOCHS)
model.train(train_Dataset,val_Dataset,
            learning_rate=config.LEARNING_RATE,
            epochs=config.EPOCHS,
            layers = 'heads',
            max_queue_size=config.MAX_QUEUE,
            use_multiprocessing=False)

===== PRINT HERE ====
20
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
rpn_model              (Functional)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4         (TimeDistributed)
mrcnn_bbox_fc          (TimeDistributed)
mrcnn_mask_deconv      (TimeDistributed)
mrcnn_class_logits     (TimeDistributed)
mrcnn_mask             (TimeDistributed)
n

/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/mask_rcnn/roi_align_classifier/sub:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/mask_rcnn/roi_align_classifier/GatherV2_2:0", shape=(None, 7, 7, 256), dtype=float32), dense_shape=Tensor("gradient_tape/mask_rcnn/roi_align_classifier/Shape_2:0", shape=(4,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/mask_rcnn/roi_align_classifier/sub_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/mask_rcnn/roi_align_classifier/GatherV2_5:0", shape=(None, 7, 7, 256), dtype=float32), dense_shape=Tensor("gradient_tape/mask_rcnn/roi_align_classifier/Shape_3